<div style="width: 100%; margin: 0 auto;">
    <a href="https://github.com/e10101/learning-operations-research">
        <img src="../assets/banner.svg" alt="Learning Operations Research" style="width: 100%; height: auto; display: block;">
    </a>
</div>

# Integer Programming - Media Selection with Piecewise Linear Functions
---

[![Github](../assets/badges/github.svg)](https://github.com/e10101/learning-operations-research)


## Problem

### Problem Details

Dorian Auto has a budget to purchase full-page ads in two magazines, Inside Jocks (IJ) and Family Square (FS), to maximize total advertising exposures. 

The number of exposures generated per ad is not constant but depends on the total number of ads placed in that specific magazine.

### Parameters

These are the given constant values and rates in the problem.

- **Total Advertising Budget**: 20,000 
- **Cost per full-page ad**: 1,000 (for both magazines)
- **Exposure rates for Inside Jocks (IJ)**: The number of exposures generated *per ad* depends on the total number of ads (n) placed in IJ.
    - 10,000 exposures/ad for 1 ≤ n ≤ 6 ads
    - 3,000 exposures/ad for 7 ≤ n ≤ 10 ads
    - 2,500 exposures/ad for 11 ≤ n ≤ 15 ads
    - 0 exposures/ad for n ≥ 16 ads
- **Exposure rates for Family Square (FS)**: The number of exposures generated *per ad* depends on the total number of ads (n) placed in FS.
    - 8,000 exposures/ad for 1 ≤ n ≤ 4 ads
    - 6,000 exposures/ad for 5 ≤ n ≤ 12 ads
    - 2,000 exposures/ad for 13 ≤ n ≤ 15 ads
    - 0 exposures/ad for n ≥ 16 ads

### Objective

Maximize the total number of advertising exposures obtained from purchasing ads in both magazines.

### Source

This problem is adapted from:

Winston, W.L. (2004). *Operations Research Applications and Algorithms* (4th ed.). Duxbury Press, Pacific Grove, CA. (Chapter 9: Integer Programming, p. 494).

## Create Model

In [95]:
import gurobipy as gp

In [96]:
m = gp.Model("Piecewise-Linear Functions")

## Create Variables

In [97]:
BREAKS = 5
MEDIAS = 2

In [98]:
X = m.addVars(MEDIAS, vtype=gp.GRB.INTEGER, lb=0, name="ads")

In [99]:
Z = m.addVars(MEDIAS, BREAKS, vtype=gp.GRB.CONTINUOUS, lb=0, name="z")

In [100]:
Y = m.addVars(MEDIAS, BREAKS-1, vtype=gp.GRB.BINARY, lb=0, name="y")

## Create Constraints

In [101]:
# X
BREAK_POINTS = [
    [0, 6, 10, 15, 20],
    [0, 4, 12, 15, 20],
]

for i in range(MEDIAS):
    m.addConstr(sum([Z[i, j] * BREAK_POINTS[i][j] for j in range(BREAKS)]) == X[i], name=f"x_{i}_{j}")

In [102]:
# Z
for i in range(MEDIAS):
    m.addConstr(sum([Z[i, j] for j in range(BREAKS)]) == 1, name=f"z_{i}")

In [103]:
# Y
for i in range(MEDIAS):
    m.addConstr(sum([Y[i, j] for j in range(BREAKS-1)]) == 1, name=f"y_{i}")

In [104]:
# Z+Y

for i in range(MEDIAS):
    for j in range(BREAKS):
        if j == 0:
            m.addConstr(Z[i, j] <= Y[i, j], name=f"z_y_{i}_{j}")
        elif j == BREAKS - 1:
            m.addConstr(Z[i, j] <= Y[i, j-1], name=f"z_y_{i}_{j}")
        else:
            m.addConstr(Z[i, j] <= Y[i, j-1] + Y[i, j], name=f"z_y_{i}_{j}")

In [105]:
# Budget
BUDGET = 20_000
PAGE_COST = 1_000

m.addConstr(sum([X[i] * PAGE_COST for i in range(MEDIAS)]) <= BUDGET, name="budget")

<gurobi.Constr *Awaiting Model Update*>

## Set Objective

In [106]:
EXPOSURES = [
    [
        0,
        60_000,
        72_000,
        84_500,
        84_500,
    ],
    [
        0,
        32_000,
        80_000,
        86_000,
        86_000,
    ],
]

In [107]:
# Obj
m.setObjective(sum([EXPOSURES[i][j] * Z[i, j] for j in range(BREAKS) for i in range(MEDIAS)]), sense=gp.GRB.MAXIMIZE)

## Solve

In [108]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (armlinux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: ARM64
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 20 columns and 56 nonzeros
Model fingerprint: 0xbc25bbb8
Variable types: 10 continuous, 10 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+04, 9e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolved: 15 rows, 18 columns, 52 nonzeros
Variable types: 10 continuous, 8 integer (6 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 1.460000e+05, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    146000.00000 146000.000  0.00% 

## Result

In [109]:
m.objVal

146000.0

In [110]:
m.getVars()

[<gurobi.Var ads[0] (value 8.0)>,
 <gurobi.Var ads[1] (value 12.0)>,
 <gurobi.Var z[0,0] (value 0.0)>,
 <gurobi.Var z[0,1] (value 0.5)>,
 <gurobi.Var z[0,2] (value 0.5)>,
 <gurobi.Var z[0,3] (value 0.0)>,
 <gurobi.Var z[0,4] (value 0.0)>,
 <gurobi.Var z[1,0] (value 0.0)>,
 <gurobi.Var z[1,1] (value 0.0)>,
 <gurobi.Var z[1,2] (value 1.0)>,
 <gurobi.Var z[1,3] (value 0.0)>,
 <gurobi.Var z[1,4] (value 0.0)>,
 <gurobi.Var y[0,0] (value 0.0)>,
 <gurobi.Var y[0,1] (value 1.0)>,
 <gurobi.Var y[0,2] (value -0.0)>,
 <gurobi.Var y[0,3] (value 0.0)>,
 <gurobi.Var y[1,0] (value 0.0)>,
 <gurobi.Var y[1,1] (value 1.0)>,
 <gurobi.Var y[1,2] (value -0.0)>,
 <gurobi.Var y[1,3] (value 0.0)>]

In [111]:
for c in m.getConstrs():
    print(c, c.Slack)

<gurobi.Constr x_0_4> 0.0
<gurobi.Constr x_1_4> 0.0
<gurobi.Constr z_0> 0.0
<gurobi.Constr z_1> 0.0
<gurobi.Constr y_0> 0.0
<gurobi.Constr y_1> 0.0
<gurobi.Constr z_y_0_0> 0.0
<gurobi.Constr z_y_0_1> 0.5
<gurobi.Constr z_y_0_2> 0.5
<gurobi.Constr z_y_0_3> 0.0
<gurobi.Constr z_y_0_4> 0.0
<gurobi.Constr z_y_1_0> 0.0
<gurobi.Constr z_y_1_1> 1.0
<gurobi.Constr z_y_1_2> 0.0
<gurobi.Constr z_y_1_3> 0.0
<gurobi.Constr z_y_1_4> 0.0
<gurobi.Constr budget> 0.0
